# Demonstration: GroupCommutatorIteration for Double-Bracket Quantum Algorithms

This notebook demonstrates how to use the `GroupCommutatorIteration` class from `boostvqe` to simulate double-bracket quantum algorithms using group commutator approximations.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qibo import gates, symbols
from qibo import hamiltonians
from qibo.hamiltonians import Hamiltonian, SymbolicHamiltonian

from boostvqe.models.dbqa.group_commutator_iteration import GroupCommutatorIteration, DoubleBracketRotationApproximationType
from boostvqe.models.dbqa.evolution_oracles_CZ_gates import EvolutionOracle, EvolutionOracleType
from boostvqe.models.dbqa.double_bracket_iteration import DoubleBracketIteration

from copy import deepcopy

/home/marek/.local/lib/python3.12/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## 1. Initialize Hamiltonian and Evolution Oracles

We define a simple two-qubit Hamiltonian and create the required EvolutionOracle objects.

In [2]:
def h_expectation_from_circ(ham, circ):
    # calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
        ham.backend.execute_circuit(circuit=circ).state())
    
# XXZ model matrix
from qibo.symbols import X, Y, Z
def construct_XXZ(nqubits, delta=0.5, boundary='closed'):
    if boundary == 'periodic' or boundary == 'closed':
        return hamiltonians.XXZ(nqubits, delta)
    elif boundary == 'open':
        H_sym = sum([X(i)*X(i+1)+ Y(i)*Y(i+1) + delta* Z(i)*Z(i+1) for i in range(nqubits-1)])
        return hamiltonians.SymbolicHamiltonian(H_sym)

def h_expectation_from_circ(ham, circ):
# calculates the exact expectation of hamiltonian given a circuit in qibo
    return ham.expectation(
    ham.backend.execute_circuit(circuit=circ).state())
    
# XXZ model matrix
from qibo.symbols import X, Y, Z
def construct_XXZ(nqubits, delta=0.5, boundary='closed'):
    if boundary == 'periodic' or boundary == 'closed':
        return hamiltonians.XXZ(nqubits, delta)
    elif boundary == 'open':
        H_sym = sum([X(i)*X(i+1)+ Y(i)*Y(i+1) + delta* Z(i)*Z(i+1) for i in range(nqubits-1)])
        return hamiltonians.SymbolicHamiltonian(H_sym)
    
hamiltonian = construct_XXZ(nqubits=4, delta=0.5, boundary='open')
# Create EvolutionOracle for the Hamiltonian
hamiltonian_oracle = EvolutionOracle(
    h=hamiltonian,
    evolution_oracle_type=EvolutionOracleType.hamiltonian_simulation
)

diagonal_oracle = deepcopy(hamiltonian_oracle)  # For demonstration, use the same oracle as diagonal

[Qibo 0.2.21|INFO|2025-07-28 12:58:44]: Using qibojit (numba) backend on /CPU:0


In [3]:
hamiltonian_oracle.h.backend


qibojit (numba)

In [4]:
type(hamiltonian)

qibo.hamiltonians.hamiltonians.SymbolicHamiltonian

## 2. Instantiate GroupCommutatorIteration

We now create an instance of `GroupCommutatorIteration` using the Hamiltonian EvolutionOracle and a chosen group commutator approximation type.

In [5]:
# Instantiate GroupCommutatorIteration with group_commutator type
gci = GroupCommutatorIteration(
    input_hamiltonian_evolution_oracle=hamiltonian_oracle,
    double_bracket_rotation_type=DoubleBracketRotationApproximationType.group_commutator
)
print(f"Number of qubits: {gci.nqubits}")

Number of qubits: 4


## 3. Run a Group Commutator Step

We run a single group commutator step and print the resulting circuit.

In [6]:
from qibo.symbols import Z

# Define a diagonal Hamiltonian with a magnetic field term (e.g., sum_i Z_i)

nqubits = gci.nqubits
magnetic_field_strength = 1.0
H_diag = magnetic_field_strength * sum([Z(i) for i in range(nqubits)])
diagonal_hamiltonian = SymbolicHamiltonian(H_diag)

# Create EvolutionOracle for the diagonal Hamiltonian
diagonal_magnetic_oracle = EvolutionOracle(
    h=diagonal_hamiltonian,
    evolution_oracle_type=EvolutionOracleType.hamiltonian_simulation
)
step_duration = 0.1  # Define the step duration for the evolution
# Execute a call to gci with the new diagonal oracle
result_circuit = gci(
    diagonal_association=diagonal_magnetic_oracle,
    step_duration=step_duration
)

print(result_circuit)

None


## 4. Optimize Step Duration

We use the `choose_step` method to find the optimal step duration over a grid and plot the loss landscape.

In [7]:
# Optimize step duration over a grid
step_grid = np.linspace(0.01, 0.5, 50)
opt_step, min_loss, losses = commutator_iter.choose_step(
    d=diagonal_oracle,
    step_grid=step_grid,
    mode_dbr=DoubleBracketRotationApproximationType.group_commutator
)

print(f"Optimal step duration: {opt_step:.4f}")
print(f"Minimum loss: {min_loss:.6f}")

plt.figure(figsize=(6,4))
plt.plot(step_grid, losses, marker='o')
plt.xlabel('Step Duration')
plt.ylabel('Loss')
plt.title('Loss Landscape for Step Duration')
plt.grid(True)
plt.show()

NameError: name 'commutator_iter' is not defined

## 5. Analyze Gate Counts

We analyze the quantum circuit's gate usage using the provided methods.

In [ ]:
# Analyze gate counts
circuit = commutator_iter.get_composed_circuit()
print(f"CNOT count: {commutator_iter.count_CNOTs(circuit)}")
print(f"CZ count: {commutator_iter.count_CZs(circuit)}")
print(f"RBS count: {commutator_iter.count_RBS(circuit)}")
print("Gate count dictionary:")
print(commutator_iter.get_gate_count_dict())

commutator_iter.print_gate_count_report()

## 6. Visualize Loss Landscape

We plot the loss as a function of step duration to visualize the optimization behavior.

In [ ]:
# The loss landscape was already plotted above in the optimization section.
# This cell is a placeholder for further visualizations if needed.